# M3: LLM Benchmark (Zero-Shot Inference)

V tomto notebooku porovnáme výkon našich natrénovaných modelů (Logistic Regression, Mahalanobis) s moderními Large Language Models (LLM) v režimu Zero-Shot.

## 1. Setup & Imports

In [2]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# Auto-reload modules for development
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Add src to path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)

# Import custom modules
import config
import data_splitting
import models
import visualization
import experiments
import evaluation
from llm_client import LLMClassifier

# Setup visualization style
visualization.setup_style()

print(f"✅ Setup complete. Data dir: {config.DATA_DIR}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2026-02-09 15:56:27,407 - INFO - 🎨 Visualization style set: whitegrid


✅ Setup complete. Data dir: C:\Users\dobes\Documents\UniversityCodingProject\ThesisCoding\data


# 1. Načtení Testovacích Dat
Musíme použít **stejnou testovací sadu** jako v M2/S2 (Sentence Supervised), aby bylo srovnání fér.

In [ ]:
# Načteme Baseline scénář (S2a)
data = data_splitting.get_train_val_test_splits(
    scenario='baseline',
    level='sentence',
    pooling='mean', # Pooling tu nehraje roli, jde nám o text
    random_state=42
)

# Nás zajímá jen TEST set a původní texty
df_test = data['meta_test'].copy()
df_test['true_label'] = data['y_test']

print(f"📊 Test Set Size: {len(df_test)} sentences")
display(df_test.head(3))

## 2. Definice LLM Modelů
Vybereme modely, které chceme testovat.
- **Gemini:** Vyžaduje GOOGLE_API_KEY v .env
- **Mistral:** Vyžaduje HF_TOKEN v .env (zdarma přes Inference API, ale může mít rate limit)

In [ ]:
MODELS_TO_TEST = [
    {
        'id': 'gemini-flash',
        'provider': 'gemini',
        'model_name': 'gemini-1.5-flash',
        'display_name': 'Gemini 1.5 Flash'
    },
    {
        'id': 'mistral-7b',
        'provider': 'huggingface',
        # Použijeme v0.3, je novější a lepší v následování instrukcí
        'model_name': 'mistralai/Mistral-7B-Instruct-v0.3', 
        'display_name': 'Mistral 7B (HF)'
    }
]

# Cesta pro ukládání výsledků
RESULTS_FILE = config.RESULTS_DIR / "M3_LLM_predictions.csv"

## 3. Inference Loop (Smyčka dotazování)
Iterujeme přes věty a modely. Průběžně ukládáme výsledky, abychom o ně nepřišli.

In [ ]:
# Pokud už soubor existuje, načteme ho, abychom neplatili/nečekali zbytečně
if RESULTS_FILE.exists():
    df_results = pd.read_csv(RESULTS_FILE)
    print(f"♻️ Načteny předchozí výsledky: {len(df_results)} řádků.")
else:
    df_results = pd.DataFrame()

# Projdeme všechny modely
for model_cfg in MODELS_TO_TEST:
    model_id = model_cfg['id']
    print(f"\n🚀 Starting Inference for: {model_cfg['display_name']}...")
    
    # Inicializace klienta
    try:
        clf = LLMClassifier(
            provider=model_cfg['provider'], 
            model_name=model_cfg['model_name']
        )
    except Exception as e:
        print(f"❌ Skipper model {model_id}: {e}")
        continue

    # Připravíme sloupec pro výsledky, pokud neexistuje
    col_name = f"pred_{model_id}"
    if col_name not in df_results.columns:
        df_results[col_name] = np.nan
        # Zkopírujeme metadata z df_test, pokud je df_results prázdný
        if df_results.empty:
            df_results = df_test.copy()
    
    # Smyčka přes věty
    # Procházíme jen ty, které ještě nemají výsledek (NaN)
    missing_mask = df_results[col_name].isna()
    indices_to_process = df_results[missing_mask].index
    
    print(f"   -> Processing {len(indices_to_process)} sentences...")
    
    for idx in tqdm(indices_to_process, desc=f"Asking {model_id}"):
        text = df_results.loc[idx, 'text']
        
        # Volání API
        prediction = clf.predict(text)
        
        # Uložení
        if prediction is not None:
            df_results.loc[idx, col_name] = prediction
        else:
            print(f"   ⚠️ Failed prediction for index {idx}")
        
        # Průběžné uložení každých 10 vět (prevence ztráty dat)
        if idx % 10 == 0:
            df_results.to_csv(RESULTS_FILE, index=False)
            
    # Finální uložení po modelu
    df_results.to_csv(RESULTS_FILE, index=False)
    print(f"✅ Model {model_id} finished.")

print("\n🏁 All benchmarks completed.")
display(df_results.head())

## 4. Vyhodnocení a Srovnání
Spočítáme metriky pro LLM a přidáme "ručně" nejlepší výsledek z M2 (LogReg), abychom měli kontext.

In [ ]:
# 1. Definice Baseline (Tvůj nejlepší model z M2/S2)
# Z těchto zadaných csv vzít nejlepší modely podle zadané matriky
# Např.:
#           M2_S2_path =  config.RESULTS_DIR / ".csv"



# 2. Výpočet metrik pro LLM
metrics_list = [BASELINE_PERFORMANCE]

for model_cfg in MODELS_TO_TEST:
    col_name = f"pred_{model_cfg['id']}"
    
    # Vynecháme řádky, kde se API nepovedlo (NaN)
    valid_rows = df_results.dropna(subset=[col_name])
    
    if len(valid_rows) == 0:
        continue
        
    y_true = valid_rows['true_label']
    y_pred = valid_rows[col_name].astype(int)
    
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    
    metrics_list.append({
        'Model': model_cfg['display_name'],
        'F1 Score': f1,
        'Accuracy': acc,
        'Type': 'LLM (Zero-Shot)'
    })
    
    print(f"\n📊 Report: {model_cfg['display_name']}")
    print(classification_report(y_true, y_pred, target_names=['Neutral', 'Bias']))

## 5. Vizualizace Srovnání
Kdo vyhrál? David (LogReg) nebo Goliáš (Gemini)?